## WL Soccer Video processing to create Dataset
**Author:** Christian Byron  **Date:** 21-Jun-21

This notebook is an interactive tool to design and build a dataset from a video file. The generic GUI `VideoDatasetUI.py` accepts a dataset object for reading the data including constructing the video. The first dataset created below reads from one list and two files:

- the video clip file `video.txt` that provides start and end times for a given segment of the video (as per the list)
- the annotations file `annotations.txt` that provides a sample annotation for one or many frames within the clip

### Step 1 - Define the Dataset class

- [ ] Clean up into its own file module

In [2]:
import os
import cv2

def open_data_file(filename, columns):
    data = []
    try:
        with open(filename) as f:
            for line in f.readlines() :
                line = line.strip() # remove leading/trailing white spaces

                d = {} # dictionary to store file data (each line)
                data_entry = [item.strip() for item in line.split(',')]
                for index, elem in enumerate(data_entry):
                        d[columns[index]] = data_entry[index]

                data.append(d) # append dictionary to list
    finally:        
        return data


def save_data_file(filename, data):

    with open(filename, 'w') as f:
        for data_entry in data:
            f.write( ', '.join(str(x) for x in data_entry.values()))
            f.write('\n')
            
            
class WLS_dataset:

        def __read_video_list_file(self):
            return open_data_file("video_clips.txt", ["FileName","Clip Name", "Start", "End"])
        
        def __init__(self):
            self.__base_video_path = "C:\\Users\\s441606\\Documents\\Videos\\"
          
            self.__video_clips = self.__read_video_list_file()
        
        def video_list(self): 
            return [clip.get("Clip Name") for clip in self.__video_clips]

        def __get_video_clip_data(self, video_ref):
            return self.__video_clips[self.video_list().index(video_ref)]

        def add_video_to_list(self, file_name, clip_name, start, end):
            self.__video_clips.append({"FileName": file_name, "Clip Name": clip_name, "Start": start, "End": end})
            
        def save_video_list_to_file(self):
            save_data_file("video_clips.txt", self.__video_clips)
        
        def video_annotations(self, video_ref):
            return []
        
        def video_annotation_frame_num(self,video_ref, annotation_ref):
            pass
        
        def video_end_frame_num(self, video_ref):
            return int(self.__get_video_clip_data(video_ref)["End"])
        
        def video_start_frame_num(self, video_ref):
            return int(self.__get_video_clip_data(video_ref)["Start"])
        
        def video_frame(self, video_ref, frame_num):
            path = self.__base_video_path + self.__get_video_clip_data(video_ref)["FileName"] 
            cap = cv2.VideoCapture(path)
            cap.set(1, frame_num)
            ret, frame = cap.read()
            is_success, im_buf_arr = cv2.imencode(".jpg", frame)
            byte_im = im_buf_arr.tobytes()
            cap.release()
            return byte_im
                                    
        def video_annotated_frame(self, video_ref, annotation_ref, frame_num):
            pass

### Step 2 - Display the Video Controls

In [3]:
from VideoDatasetUI import VideoDatasetUI

ds = WLS_dataset()
videoUI = VideoDatasetUI(ds)
videoUI.display()

{'new': 0}


{'name': 'value', 'old': 41, 'new': 42, 'owner': IntSlider(value=42, max=83), 'type': 'change'}
{'name': 'value', 'old': 83, 'new': 82, 'owner': IntSlider(value=82, max=83, min=42), 'type': 'change'}
{'name': 'value', 'old': 82, 'new': 58, 'owner': IntSlider(value=58, max=83, min=42), 'type': 'change'}
{'name': 'value', 'old': 58, 'new': 52, 'owner': IntSlider(value=52, max=83, min=42), 'type': 'change'}
{'name': 'value', 'old': 52, 'new': 51, 'owner': IntSlider(value=51, max=83, min=42), 'type': 'change'}
{'name': 'value', 'old': 51, 'new': 47, 'owner': IntSlider(value=47, max=83, min=42), 'type': 'change'}
{'name': 'value', 'old': 47, 'new': 46, 'owner': IntSlider(value=46, max=83, min=42), 'type': 'change'}
{'name': 'value', 'old': 46, 'new': 45, 'owner': IntSlider(value=45, max=83, min=42), 'type': 'change'}
{'name': 'value', 'old': 45, 'new': 44, 'owner': IntSlider(value=44, max=83, min=42), 'type': 'change'}
{'name': 'value', 'old': 44, 'new': 42, 'owner': IntSlider(value=42, max

### Step 3 - Add to the video clip data to split video into components

In [1]:
import ipywidgets as widgets

video_file_dropdown = widgets.Dropdown( options=['IMG_0085.MOV', 'IMG_0087.MOV'],
                                        layout=widgets.Layout(width='160px'),
                                        disabled=False,
                                       )

video_clip_name = widgets.Text( placeholder='Enter here',
                                layout=widgets.Layout(width='160px'),
                                disabled=False
                               )

video_clip_start = widgets.IntText(value=0, disabled=False, layout=widgets.Layout(width='80px'))

video_clip_end = widgets.IntText(value=0, disabled=False, layout=widgets.Layout(width='80px'))

save_btn = widgets.Button( description='Save', disabled=False)

def on_save_btn_click(btn):
    ds.add_video_to_list(video_file_dropdown.value, video_clip_name.value,
                         video_clip_start.value, video_clip_end.value)
    ds.save_video_list_to_file()

save_btn.on_click(on_save_btn_click)

widgets.HBox( [widgets.Label('Video File:'),video_file_dropdown, 
              widgets.Label('Clip Name:'), video_clip_name, 
              widgets.Label('Start:'), video_clip_start, 
              widgets.Label('End:'),video_clip_end, save_btn])